# TECHNIKI PROGRAMOWANIA - projekt 3

## Krzysztof Toczyński 197986

## Lucjan Butko 198090  
     
    Wykorzystane biblioteki:
    - matplotplusplus (c++)
    - audiofile(c++)
    - scikit_build_example(c++) ["pybind11/pybind11.h"]
    - vector(c++) [STL]
    - cmath(c++) [STL]
    -complex(c++) [STL]

Cały plik main.cpp dostępny w repozytorium https://github.com/TofTo23/pro_3.git
src/main.cpp. Przed użyciem należy zmienić ścieżkę set(pybind11_DIR 'lokalizacja/biblioteki/pybind11') w CMakeLists.txt.

## 1.Wizualizacja sygnału z wykorzystaniem biblioteki matplotplusplus



Kod w c++
```c++
AudioFile<double> audioFile;
string filePath = "taco1.wav";  // Taco Hemingway

if (!audioFile.load(filePath)) {
    cerr << "Nie można wczytać pliku audio: " << filePath << endl;
}

double T = audioFile.getLengthInSeconds();

vector<double> t;
t = linspace(0, T, (T * fs));

vector<double> y;
y.reserve(fs);

for (int i = 0; i < fs; ++i) {
    y.push_back(audioFile.samples[0][i]); // [0] - tylko jeden kanał 
}

// --- wykres dla fs sampli
auto w1 = figure();
w1->size(800, 600);
plot(t, y);
title("Audio");
xlabel("Próbki");
ylabel("Amplituda");
show();
```

In [ ]:
import scikit_build_example
scikit_build_example.display_audio()

<img src="1_audio.png" />

## 2.DFT i transformata odwrotna

### a) dla sinusa, cosinusa, sygnału prostokątnege oraz sygnału piłokształtnego

Kod w c++
```c++
     vector<complex<double>> dft(y.size());
 for (size_t i = 0; i < y.size(); ++i) {
     complex<double> sum(0.0, 0.0);
     for (size_t j = 0; j < y.size(); ++j) {
         double kat = 2 * pi * i * j / y.size();
         sum += y[j] * polar(1.0, -kat); //zespolak 
     }
     dft[i] = sum;
 }

 vector<double> Adft(y.size());
 for (int i = 0; i < y.size(); ++i) {
     Adft[i] = abs(dft[i]);
 }

 vector<double> frq(y.size());
 for (int i = 0; i < y.size(); ++i) {
     frq[i] = i * (fs / y.size());
 }

 double n = y.size();
 vector<double> Idft(y.size());
 for (size_t i = 0; i < y.size(); ++i) {
     complex<double> sum(0.0, 0.0);
     for (size_t j = 0; j < y.size(); ++j) {
         double kat = 2 * pi * i * j / y.size();
         sum += dft[j] * polar(1.0, kat);
     }
     Idft[i] = sum.real() / n;
 }

 auto w2 = figure();
 w2->size(800, 600);
 plot(frq, Adft);
 title("DFT");
 xlabel("Częstotliwość");
 ylabel("Amplituda");

 auto w3 = figure();
 w3->size(800, 600);
 plot(t, Idft);
 title("IDFT");
 xlabel("Czas");
 ylabel("Amplituda");
 show();
```

In [1]:
#do generowania DFT oraz IDFT poszczególnych przbiegów należy za pierwszy argument przyjąć: 
# "s - Sinus"
# "c - Cosinus"
# "q - Square wave"
# "t - Sawtooth signal"
# drugi argument jest częstotliwością danego sygnału (przy audio czestotliwosc nie ma znaczenia)
import scikit_build_example
scikit_build_example.DFT_IDFT('s', 5)

<img src="2_dft.png" />

<img src="2_idft.png" />

In [1]:
#do generowania DFT oraz IDFT poszczególnych przbiegów należy za pierwszy argument przyjąć: 
# "s - Sinus"
# "c - Cosinus"
# "q - Square wave"
# "t - Sawtooth signal"
# drugi argument jest częstotliwością danego sygnału (przy audio czestotliwosc nie ma znaczenia)
import scikit_build_example
scikit_build_example.DFT_IDFT('t', 15)

<img src="2_dft_t.png" />

<img src="2_idft_t.png" />

### b) dla pliku dźwiękowego

kod w c++
```c++
void Audio_DFT_IDFT() {
    AudioFile<double> audioFile;
    string filePath = "taco1.wav";  // Taco Hemingway
    if (!audioFile.load(filePath)) {
        cerr << "Nie można wczytać pliku audio: " << filePath << endl;
    }
    double T = audioFile.getLengthInSeconds(); // czas trwania audio

    vector<double> t;
    t = linspace(0, T, (T * fs));

    vector<double> y;
    y.reserve(fs);
    for (int i = 0; i < fs; ++i) {
        y.push_back(audioFile.samples[0][i]); // [0] - tylko jeden kanał 
    }

    double n = y.size();
    vector<complex<double>> dft(n);
    for (size_t i = 0; i < n; ++i) {
        complex<double> sum(0.0, 0.0);
        for (size_t j = 0; j < n; ++j) {
            double kat = 2 * pi * i * j / n;
            sum += y[j] * polar(1.0, -kat); //zespolak 
        }
        dft[i] = sum;
    }

    vector<double> Adft(n);
    for (int i = 0; i < n; ++i) {
        Adft[i] = abs(dft[i]);
    }

    vector<double> frq(n);
    for (int i = 0; i < n; ++i) {
        frq[i] = i * (fs / n);
    }

    vector<double> Idft(n);
    for (size_t i = 0; i < n; ++i) {
        complex<double> sum(0.0, 0.0);
        for (size_t j = 0; j < n; ++j) {
            double kat = 2 * pi * i * j / n;
            sum += dft[j] * polar(1.0, kat);
        }
        Idft[i] = sum.real() / fs;
    }

    auto w2 = figure();
    w2->size(800, 600);
    plot(frq, Adft);
    title("DFT");
    xlabel("Częstotliwość");
    ylabel("Amplituda");

    auto w3 = figure();
    w3->size(800, 600);
    plot(t, Idft);
    title("IDFT");
    xlabel("Czas");
    ylabel("Amplituda");
    show();
}
```

In [ ]:
import scikit_build_example as sc
sc.Audio_DFT_IDFT()

<img src="2_dft_audio.png" />

<img src="2_idft_audio.png" />

## 4.Generowanie sygnałów o zadanej częstotliwości (sin, cos, prostokątny, piłokształtny)

### a) sinus w c++

```c++
void display_sinus(double T, double F, double A) {

    vector<double> t = linspace(0, T, (T * fs));

    vector<double> y(t.size());
    for (size_t i = 0; i < t.size(); ++i) {
        y[i] = A * sin(2 * pi * F * t[i]);
    }

    plot(t, y);
    title("Sinus");
    xlabel("t(s)");
    ylabel("y");
    show();
}
```

In [4]:
#wywołanie funkcji sinus w pythonie, gdzue argumenty funkcji to odpowiednio: okres, częstotliwość, amplituda
import scikit_build_example
scikit_build_example.display_sinus(1, 1, 1)

<img src="4_sinus.png" />

### b) cosinus w c++

```c++
void display_cosinus(double T, double F, double A) {

    vector<double> t = linspace(0, T, (T * fs));

    vector<double> y(t.size());
    for (size_t i = 0; i < t.size(); ++i) {
        y[i] = A * cos(2 * pi * F * t[i]);
    }

    plot(t, y);
    title("Cosinus");
    xlabel("t(s)");
    ylabel("y");
    show();
}
```

In [2]:
#wywołanie funkcji cosinus w pythonie, gdzue argumenty funkcji to odpowiednio: okres, częstotliwość, amplituda
import scikit_build_example
scikit_build_example.display_cosinus(2, 0.3, 4)

<img src="4_cosinus.png" />

### c) prostokątny w c++

```c++
void display_squarewave(double T, double F, double A) {

    vector<double> t = linspace(0, T, (T * fs));

    vector<double> y(t.size());
    for (size_t i = 0; i < t.size(); ++i) {
        y[i] = A * sin(2 * pi * F * t[i]) > 0 ? A : -A;
    }

    plot(t, y);
    title("Square wave");
    xlabel("t(s)");
    ylabel("y");
    show();
}
```

In [6]:
#wywołanie funkcji squarewave w pythonie, gdzue argumenty funkcji to odpowiednio: okres, częstotliwość, amplituda
import scikit_build_example
scikit_build_example.display_squarewave(0.5, 45, 2)

<img src="4_squarewave.png" />

### d) piłokształtny w c++

```c++
void display_sawtooth(double T, double F, double A) {

    vector<double> t = linspace(0, T, (T * fs));

    vector<double> y(t.size());
    for (size_t i = 0; i < t.size(); ++i) {
        y[i] = fmod(F * t[i] + 0.5, 1) * (2 * A) - A;
    }

    plot(t, y);
    title("Sawtooth wave");
    xlabel("t(s)");
    ylabel("y");
    show();
}

```

In [7]:
#wywołanie funkcji sawtooth w pythonie, gdzue argumenty funkcji to odpowiednio: okres, częstotliwość, amplituda
import scikit_build_example
scikit_build_example.display_sawtooth(1, 2, 0.3)

<img src="4_sawtooth.png" />

## 6.Usuwanie niskich częstotliwości z sygnału (korzystajac z DFT)

### a) dla sinusa, cosinusa, sygnału prostokątnege oraz sygnału piłokształtnego

Kod w c++
```c++
 vector<complex<double>> dft(y.size());
 for (size_t i = 0; i < y.size(); ++i) {
     complex<double> sum(0.0, 0.0);
     for (size_t j = 0; j < y.size(); ++j) {
         double kat = 2 * pi * i * j / y.size();
         sum += y[j] * polar(1.0, -kat);
     }
     dft[i] = sum;
 }

 size_t syllabus = static_cast<size_t>(uf * dft.size() / fs);

 for (int i = 0; i < syllabus; ++i) {
     dft[i] = 0.0;
 }

 vector<double> cutted(dft.size());
 double n = y.size();
 for (size_t i = 0; i < y.size(); ++i) {
     complex<double> sum(0.0, 0.0);
     for (size_t j = 0; j < y.size(); ++j) {
         double kat = 2 * pi * i * j / y.size();
         sum += dft[j] * polar(1.0, kat);
     }
     cutted[i] = sum.real() / n;
 }

 auto w4 = figure();
 w4->size(800, 600);
 plot(t, y);
 title("Oryginalny sygnał");
 xlabel("Czas");
 ylabel("Amplituda");

 auto w5 = figure();
 w5->size(800, 600);
 plot(t, cutted);
 title("Sygnał po usunięciu niskich częstotliwości");
 xlabel("Czas");
 ylabel("Amplituda");

 show();
```

In [6]:
#do usuwania niskich częstotliwości za pomocą DFT oraz IDFT poszczególnych przbiegów należy za pierwszy argument przyjąć: 
# "s - Sinus"
# "c - Cosinus"
# "q - Square wave"
# "t - Sawtooth signal"
# drugi argument jest częstotliwością danego sygnału (przy audio czestotliwosc nie ma znaczenia)
# trzeci argument to częstotliwośc graniczna 
import scikit_build_example
scikit_build_example.usuwanie('s', 3, 150)

### b) dla pliku dźwiękowego

kod w c++
```c++
     AudioFile<double> audioFile;
 string filePath = "taco1.wav";  // Taco Hemingway
 if (!audioFile.load(filePath)) {
     cerr << "Nie można wczytać pliku audio: " << filePath << endl;
 }
 double T = audioFile.getLengthInSeconds(); // czas trwania audio

 vector<double> t;
 t = linspace(0, T, (T * fs));

 vector<double> y;
 y.reserve(fs);
 for (int i = 0; i < fs; ++i) {
     y.push_back(audioFile.samples[0][i]); // [0] - tylko jeden kanał 
 }

 double n = y.size();
 vector<complex<double>> dft(n);
 for (size_t i = 0; i < n; ++i) {
     complex<double> sum(0.0, 0.0);
     for (size_t j = 0; j < n; ++j) {
         double kat = 2 * pi * i * j / n;
         sum += y[j] * polar(1.0, -kat); //zespolak 
     }
     dft[i] = sum;
 }

 size_t syllabus = static_cast<size_t>(uf * dft.size() / fs);

 for (int i = 0; i < syllabus; ++i) {
     dft[i] = 0.0;
 }

 vector<double> cutted(dft.size());
 for (size_t i = 0; i < n; ++i) {
     complex<double> sum(0.0, 0.0);
     for (size_t j = 0; j < n; ++j) {
         double kat = 2 * pi * i * j / n;
         sum += dft[j] * polar(1.0, kat);
     }
     cutted[i] = sum.real() / n;
 }

 auto w4 = figure();
 w4->size(800, 600);
 plot(t, y);
 title("Oryginalny sygnał");
 xlabel("Czas");
 ylabel("Amplituda");

 auto w5 = figure();
 w5->size(800, 600);
 plot(t, cutted);
 title("Sygnał po usunięciu niskich częstotliwości");
 xlabel("Czas");
 ylabel("Amplituda");

 show();
}
```

In [ ]:
import scikit_build_example
scikit_build_example.Audio_usuwanie(4000)

<img src="6_audio.png" />

<img src="6_usuwanie_audio.png" />